In [1]:
from ai_data_eng.searching.graph import *
from ai_data_eng.searching.utils import *
from ai_data_eng.searching.searchning import * 
from ai_data_eng.searching.a_star_time_opt import * 
from ai_data_eng.searching.heuristics import WeightedAverageTimeHeuristic
from pathlib import Path

DATA_DIR = Path('../data')

In [2]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])
g = Graph(connection_graph, add_constant_change_time)

C:\Users\julia\AppData\Local\Temp\ipykernel_29216\1218945369.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [3]:
from typing import Set
from queue import PriorityQueue


def judge_t_solution(solution, visiting_stops: Set[str]):
    matched_stops = 0 
    conn_time = diff(solution[-1]['arrival_sec'], solution[-1]['departure_sec'])
    line_changes = np.sum([1 for (c1, c2) in zip(solution[1:], solution[:-1]) if c1['line'] != c2['line']])
    
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return conn_time / (matched_stops + 0.01)    
    
def naive_solution(start_stop: str, visiting_stops: List[str], leave_hour: str):
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        solution += sub_sol
        prev_stop = stop 
        # add change time 
        leave_hour = sec_to_time(sub_sol[-1]['arrival_sec'])
    solution += a_star_time_opt(prev_stop, start_stop, leave_hour, WeightedAverageTimeHeuristic())[1]
    return solution


def tabu_search(start_stop: str, visiting_stops: List[str], leave_hour: str, criterion: OptimizationType):
    
    dep_time = time_to_normalized_sec(leave_hour)
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        solution += sub_sol
        prev_stop = stop 
        # add change time 
        leave_hour = sec_to_time(sub_sol[-1]['arrival_sec'])
    solution += a_star_time_opt(prev_stop, start_stop, leave_hour, WeightedAverageTimeHeuristic())[1]
        
    graph, travel_solution = a_star_time_opt(start_stop, visiting_stops[-1], leave_hour, WeightedAverageTimeHeuristic())
    _, return_solution = a_star_time_opt(visiting_stops[-1], start_stop, sec_to_time(travel_solution[-1]['arrival_sec']), WeightedAverageTimeHeuristic())
    
    solution = travel_solution + return_solution
    print_path(solution)
    cost = judge_t_solution(solution, set(visiting_stops))
    print(cost)

In [4]:
print_path(naive_solution('PL. GRUNWALDZKI', ['most Grunwaldzki', 'Poczta Główna', 'Renoma'], '08:00:00'))

TypeError: a_star_time_opt() missing 1 required positional argument: 'change_time'

In [ ]:
print_path(naive_solution('Tramwajowa', ['most Grunwaldzki', 'Poczta Główna', 'EPI', 'Sowia', 'Dworzec Główny (Dworcowa)', 'Rynek'], '08:00:00'))

In [ ]:
def is_neighbour(dep_time, new_solution):
    return assert_connection_path(dep_time, new_solution)

def naive_search(solution, leave_hour, start_stop, visiting_stops: List[str]):
    prev_stop = start_stop
    new_solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        new_solution = new_solution + sub_sol
        prev_stop = 

def replace_connection(solution, dep_time, stop_from, stop_to):
    
    is_neigh = False
    while not is_neigh:
        graph, travel_solution = a_star_time_opt(stop_from, stop_to, dep_time, WeightedAverageTimeHeuristic())